# Exo Lesson 5

En repartant du dataset "people.csv" (cf. lesson4) dans sa version "clean" finale:
- mettre la colonne `inactive` à `true` pour tous les users dont le `last_seen` date d'au moins un an
- avec une regex: filtrer les numéros de téléphone invalides
- ajouter une colonne indiquant si le numéro de tel correspond à un téléphone portable (06/07)
- ajouter une colonne indiquant si les coordonnées GPS de l'utilisateur correspondent bien au "country"

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
people = pd.read_csv('people.csv',sep=',')
people.head()

,id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"€98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert


In [4]:
people.shape

(209, 15)

In [5]:
def clean_people(df):
    # rename columns:
    df = df.rename(columns={'email address': 'email'})
    
    # remove rows which have an empty "first_name" (NA):
    #df = df[df.first_name.notna()] <- equivalent to next line:
    df = df.dropna(subset=['first_name'])
    
    # drop duplicates on ID column:
    df = df.drop_duplicates()
    
    # Normalize gender column:
    df['gender'] = df['gender'].replace({'Female': 'F', 'Male': 'M'})
    
    # Convert column "age" to number (coerce: put NaN for bad values):
    df['age'] = pd.to_numeric(df.age, errors='coerce')
    
    # Convert columns to date type:
    df['registration'] = pd.to_datetime(df.registration)
    df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
    # When missing, last seen should fallback to the registration date:
    df['last_seen'] = df.last_seen.combine_first(df.registration)
    
    # Add a "full_name" column by concatenating two other ones:
    df['full_name'] = df.first_name + " " + df.last_name
    
    # Add a "country" column by extracting it from the address, with a split:
    df['country'] = df.address.str.split(', ').str[1]
    
    # Column "money" contains values like "$50.23" or "€23,09".
    # We want to make it uniform (only dollar currency) and as number, not str.
    df['currency'] = df.money.str[0]  # extract first char ($/€) to a new "currency" column
    df['money'] = df.money.str[1:].str.replace(',', '.')  # extract remaining chars and replace , by .
    df['money'] = pd.to_numeric(df.money)  # convert to number
    # convert euros cells to dollar:
    df.loc[df.currency == '€', 'money'] = df[df.currency == '€'].money * 1.10
    del df['currency']  # remove "currency" column which is now useless
    
    # Keep only rows where email is not NA:
    df = df.dropna(subset=['email'])
    # Keep only rows where email is a good email:
    # CAUTION: in the real world you should not use dummy regexes like this to validate email addresses,
    # but instead use a dedicated tool like https://github.com/syrusakbary/validate_email.
    df = df[df.email.str.contains('.+@[0-9a-zA-Z\.\-_]+\.\w{2,}')]
    # Some users may use email alias (example: john.smith+truc@gmail.com is an alias for john.smith@gmail.com).
    # We want to drop these duplicates. To do that, we extract the 'alias' part with a regex:
    groups = df.email.str.extract('([0-9a-zA-Z\.\-_]+)(\+[0-9a-zA-Z\.\-_]+)?(@[0-9a-zA-Z\.\-_]+\.\w{2,})')
    df['email'] = groups[0] + groups[2]  # we override the email with the email without the alias part
    # Then, just use drop_duplicates, which will keep the first line by default:
    df = df.drop_duplicates(subset=['email'])
    
    return df

people_clean = clean_people(people)

In [6]:
people_clean.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.180,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
1,27570,Ruthi,Ross,rross3w@sohu.com,F,57.0,20.370,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,Ruthi Ross,Poland
2,27572,Silas,Stourton,silas.stourton3y@answers.com,M,22.0,36.289,118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,Silas Stourton,China
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64.0,108.823,121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,Roxine Pettecrew,Indonesia
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.260,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic


In [7]:
people_clean.shape

(191, 17)

On met la colonne inactive à `True` pour tous les utilisateurs dont la dernière visite date de plus d'un an.

In [12]:
people_clean['inactive'] = people_clean['last_seen'] < datetime.strptime('2018-10-23', '%Y-%m-%d')

In [13]:
people_clean = people_clean.dropna(subset = ['phone'])

On filtre les numéros de téléphone invalides grâce à une REGEX.

In [17]:
people_clean = people_clean[people_clean['phone'].apply(str).str.contains('^0\d{9}$')]
people_clean.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.18,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.26,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.50,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria


On ajoute une colonne pour savoir si le numéro de tel est un portable (06 ou 07).

In [18]:
def isMobile(x):
    if x[:2] == "06":
        return True
    elif x[:2] == "07":
        return True
    else:
        return False

In [20]:
people_clean['isMobile'] = people_clean['phone'].apply(isMobile)

In [21]:
people_clean.head()

,id,first_name,last_name,email,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,full_name,country,isMobile
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,F,63.0,55.18,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,Leandra Pabelik,Czech Republic,False
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,M,60.0,47.26,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,Gordie Bodicum,Czech Republic,False
12,27548,Stanley,McCullen,smccullen3a@timesonline.co.uk,M,60.0,NaN,119.141473,28.074649,0875503094,2017-06-05,True,2017-06-05 00:00:00,"Longquan, China",entrée/plat/dessert,Stanley McCullen,China,False
15,27619,Shelley,Skeemor,sskeemor59@princeton.edu,F,44.0,41.50,111.878471,-7.584917,0547556034,2019-01-24,False,2019-05-04 13:18:24,"Guyangan, Indonesia",entrée/plat/dessert,Shelley Skeemor,Indonesia,False
16,27442,Mort,Midford,mmidfordc@yale.edu,M,59.0,NaN,6.848501,5.959253,0811321968,2019-06-25,False,2019-06-25 00:00:00,"Ozubulu, Nigeria",entrée/plat,Mort Midford,Nigeria,False
